# Workflow
## 1. Define testbed
## 2. Define Scenario
## 3. Run Scenario on Testbed

# 1. Define Testbed

In [1]:
import sys
sys.path.append('../')
from testbed import *


def get_connection(ip='', username='', password=''):
    con = connection('_ssh')
    con.add_attribute('_ip',ip)
    con.add_attribute('_username',username)
    con.add_attribute('_password',password)
    return con


def get_testbed():

    # Device
    device_server = device(id='prince_device',type='_server')
    device_server.add_os(osHeliot('_linux'))
    device_server.add_connection(get_connection('127.0.0.1','prince', 'he@35!saFE'))
    
    my_testbed = testbed(name='prince_testbed')

    # Adding devices
    my_testbed.add_device(device_server)

    return my_testbed

In [2]:
my_testbed = get_testbed()

Adding device  prince_device  to testbed


## Validating the testbed

In [3]:
# 1) Test the reachability of devices
# 2) Download the Heliot repo from github on devices

# 3) Starts the Heliot slaves on all the devices. 
## (Provides the RPC functionality to run Tasks)

# In Andriod and other non-linux/window based devices, 
# assumption is these are setup by the user. Eg: using Android client of Heliot

my_testbed.validate()

2019-12-03 20:25:09,810 - testbed - INFO - Validating the devices in Testbed
2019-12-03 20:25:11,848 - testbed - INFO - prince_device is connected
2019-12-03 20:25:12,427 - testbed - INFO - heliot_runtime directory created
2019-12-03 20:25:14,445 - testbed - INFO - Heliot repo downloaded from Github
2019-12-03 20:25:14,447 - testbed - INFO - heliot initalization done for prince_device
2019-12-03 20:25:14,447 - testbed - INFO - Testbed validated


# 2. Defining the Scenario

## Nodes

In [4]:
from scenario import *
myScenario = scenario(name='Prince_Simple_Scenario')

server_node_1 = node(type='_server', id = 'n1')
myScenario.add_node(server_node_1)

server_node_2 = node(type='_server', id = 'n2')
myScenario.add_node(server_node_2)

server_node_3 = node(type='_server', id = 'n3')
myScenario.add_node(server_node_3)

server_node_4 = node(type='_server', id = 'n4')
myScenario.add_node(server_node_4)

virtual_switch = infranode(type='_switch', id='vs1')
myScenario.add_infranode(virtual_switch)

Adding node  n1  to scenario
Adding node  n2  to scenario
Adding node  n3  to scenario
Adding node  n4  to scenario
Adding infranode  vs1  to scenario


## Links between Nodes

In [5]:
link_n1_vswitch= mininetLink(name='link_n1_vswitch', id_1='n1',id_2='vs1')
link_n1_vswitch.add_attribute('_latency', '2') #latency in milli seconds

link_n2_vswitch= mininetLink(name='link_n2_vswitch', id_1='n2',id_2='vs1')
link_n2_vswitch.add_attribute('_latency', '2')

link_n3_vswitch= mininetLink(name='link_n3_vswitch', id_1='n3',id_2='vs1')
link_n3_vswitch.add_attribute('_latency', '2')

link_n4_vswitch= mininetLink(name='link_n4_vswitch', id_1='n4',id_2='vs1')
link_n4_vswitch.add_attribute('_latency', '2')

In [6]:
myScenario.add_mininetLink(link_n1_vswitch)
myScenario.add_mininetLink(link_n2_vswitch)
myScenario.add_mininetLink(link_n3_vswitch)
myScenario.add_mininetLink(link_n4_vswitch)

Adding mininetLink  link_n1_vswitch  to scenario
Adding mininetLink  link_n2_vswitch  to scenario
Adding mininetLink  link_n3_vswitch  to scenario
Adding mininetLink  link_n4_vswitch  to scenario


## Tasks and Dataflow between Tasks

In [7]:
task1_sendNum = taskHeliot(_type='_compute', _taskid='task1', _file='task1', _nodeid='n1')
task2_sendNum = taskHeliot(_type='_compute', _taskid='task2', _file='task2', _nodeid='n2')
task3_add = taskHeliot(_type='_compute', _taskid='task3', _file='task3', _nodeid='n3')
task4_mult = taskHeliot(_type='_compute', _taskid='task4', _file='task4', _nodeid='n4')

task1_data = task1_sendNum.get_output('task1_data')
task2_data = task2_sendNum.get_output('task2_data')
task3_add.set_input([task1_data, task2_data])
task4_mult.set_input([task1_data, task2_data])

task1_data : is set as output for task:  task1
task2_data : is set as output for task:  task2
task1_data : is set as input for task: task3
task2_data : is set as input for task: task3
task1_data : is set as input for task: task4
task2_data : is set as input for task: task4


In [8]:
myScenario.add_task(task1_sendNum)
myScenario.add_task(task2_sendNum)
myScenario.add_task(task3_add)
myScenario.add_task(task4_mult)

Adding task: task1
Adding task: task2
Adding task: task3
Adding task: task4


In [9]:
## Node mapping and ## Task mapping are implicit here based on 
## Node_types and node_ids in the task definition

# 3. Run Scenario on Testbed

In [10]:
my_testbed.set_scenario(myScenario)

Adding scenario to the testbed


In [11]:
# We need mapping of nodes and devices
# We also need mapping of tasks and nodes

## Setting up the network topology as per Scenario

In [12]:
# my_testbed.start_network()

In [13]:
# for host in my_testbed._net._hosts:
#     print(host + ":", my_testbed._net._hosts[host].IP())
    
# my_testbed._net._switches['vs10'].IP()

## Starting the Scenario Tasks

## Note: Working on task scheduling

In [14]:
my_testbed.map_task(task3_add)
my_testbed.map_task(task4_mult)
my_testbed.map_task(task1_sendNum)
my_testbed.map_task(task2_sendNum)

my_testbed.dataflow_mapping('../mapping.json')

Mapping Task: task3 , on node: n3 , mapped to device: prince_device
Mapping Task: task4 , on node: n4 , mapped to device: prince_device
Mapping Task: task1 , on node: n1 , mapped to device: prince_device
Mapping Task: task2 , on node: n2 , mapped to device: prince_device
{'prince_device': {'ip': '127.0.0.1'}, 'task3': {'in_port': 8000, 'out_ports': {}, 'device': 'prince_device', 'out_devices': {}}, 'task4': {'in_port': 8001, 'out_ports': {}, 'device': 'prince_device', 'out_devices': {}}, 'task1': {'in_port': None, 'out_ports': {'task1_data': []}, 'device': 'prince_device', 'out_devices': {'task1_data': []}}, 'task2': {'in_port': None, 'out_ports': {'task2_data': []}, 'device': 'prince_device', 'out_devices': {'task2_data': []}}}


In [15]:
#my_testbed.start_tasks()

In [16]:
# my_testbed.start_task(task3_add)
# my_testbed.start_task(task4_mult)

In [17]:
# from time import sleep
# sleep(2)

my_testbed.start_task(task1_sendNum)
# my_testbed.start_task(task2_sendNum)

Attempting to start Task: task1 , on node: n1 , mapped to device: prince_device
Attempting to start Task: task1 , mapped to device: prince_device


2019-12-03 20:25:16,714 - testbed - INFO - prince_device is connected


Running task1


In [ ]:
wait = input()

In [ ]:
# my_testbed.stop_network()